[WordAndDoc2vec](https://github.com/darecophoenixx/wordroid.sblo.jp/wiki/WordAndDoc2vec)

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
plt.style.use('seaborn')

import seaborn as sns

%config InlineBackend.figure_format = 'retina'

In [ ]:
img_cnt = 0
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.png'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name)

# load sample data

In [ ]:
X_df = pd.read_csv('../input/sample-data-wordanddoc2vec/sample002.csv', index_col=0)
X_df_user_group = pd.read_csv('../input/sample-data-wordanddoc2vec/sample002_user_group.csv', index_col=0)
X_df_brand_group = pd.read_csv('../input/sample-data-wordanddoc2vec/sample002_brand_group.csv', index_col=0)
X_df

In [ ]:
X_df_user_group

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(X_df.values.T)
save_img()

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, Input
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.constraints import max_norm

num_feature = 2

model = Sequential()
model.add(Dense(12, input_shape = (200,), activation="elu"))
model.add(Dense(8, activation="elu"))
model.add(Dense(4, activation="elu"))
model.add(Dense(num_feature, name='out2', activation="tanh"))
model.add(Dense(200, activation="linear"))

model.summary()

In [ ]:
model.get_layer('out2').get_weights()[0]

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001), metrics = ["binary_accuracy"])

In [ ]:
%%time
hst = model.fit(X_df.values, X_df.values, epochs=150,
                batch_size=128, verbose=0)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst.history["loss"]))), hst.history["loss"], label="Train loss")
ax[0].legend()
ax[1].set_title('accuracy')
ax[1].plot(list(range(len(hst.history["loss"]))), hst.history["binary_accuracy"], label="accuracy")
ax[1].legend()
save_img()

In [ ]:
output = model.get_layer("out2").output
extr = Model(model.input, output)

In [ ]:
X_proj = extr.predict(X_df.values)
X_proj.shape

proj = pd.DataFrame(X_proj)
proj.columns = ["comp_"+str(ii+1) for ii in range(num_feature)]
proj["cls"] = X_df_user_group.iloc[:,0].values
proj

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
sns.lmplot("comp_1", "comp_2", data=proj, fit_reg=False, height=8)
save_img()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
sns.lmplot("comp_1", "comp_2", data=proj, hue='cls', fit_reg=False, height=8)
save_img()

In [ ]:
sns.pairplot(proj.iloc[:,:5], hue='cls', plot_kws=dict(marker="."))
save_img()

In [ ]:
proj.to_csv('res.csv')